#### 참고) 가상환경 setting
Python 3.6.4, Keras 2.3.1, Tensorflow 2.0

In [51]:
import sys

from pathlib import Path
import numpy as np
import pandas as pd
from dateutil import parser
from datetime import timedelta

import tensorflow as tf
from tensorflow.keras import models, layers

import tqdm
from tqdm.notebook import trange

### Data import

In [2]:
Path.cwd() # 여기 나오는 경로에 파일 넣어야됨!

WindowsPath('C:/Users/KIM MIN JEONG/Desktop/Final Project')

In [3]:
TRAIN_DATASET = sorted([x for x in Path("data/HAI 2.0/training").glob("*.csv")])
TRAIN_DATASET

[WindowsPath('data/HAI 2.0/training/train1.csv'),
 WindowsPath('data/HAI 2.0/training/train2.csv'),
 WindowsPath('data/HAI 2.0/training/train3.csv')]

In [4]:
TEST_DATASET = sorted([x for x in Path("data/HAI 2.0/testing").glob("*.csv")])
TEST_DATASET

[WindowsPath('data/HAI 2.0/testing/test1.csv'),
 WindowsPath('data/HAI 2.0/testing/test2.csv'),
 WindowsPath('data/HAI 2.0/testing/test3.csv'),
 WindowsPath('data/HAI 2.0/testing/test4.csv')]

In [5]:
VALIDATION_DATASET = sorted([x for x in Path("data/HAI 2.0/validation").glob("*.csv")])
VALIDATION_DATASET

[WindowsPath('data/HAI 2.0/validation/validation.csv')]

#### Train Set merge

In [6]:
def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x.as_posix()) for x in targets])


In [7]:
TRAIN_DF_RAW = dataframe_from_csvs(TRAIN_DATASET)
TRAIN_DF_RAW

,time,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,2020-07-11 00:00:00,395.19528,12,10,52.80456,-1.2648,-1.87531,779.59595,28.02645,10832.0,...,808.29620,0.0,1.36810,8.79882,35.43700,12.01782,305.03113,301.35992,33.6555,6.0951
1,2020-07-11 00:00:01,395.14420,12,10,52.78931,-1.3147,-1.88294,780.67328,28.02473,10984.0,...,819.16809,0.0,1.36810,8.78811,35.45227,12.01782,304.27161,297.43567,33.6555,5.9262
2,2020-07-11 00:00:02,395.14420,12,10,52.79694,-1.4032,-1.88294,780.06574,28.02817,11120.0,...,823.51697,0.0,1.36734,8.81787,35.45227,12.01782,303.89179,298.66534,33.6555,5.8101
3,2020-07-11 00:00:03,395.19528,12,10,52.79694,-1.6074,-1.88294,780.15265,28.02301,11256.0,...,823.95172,0.0,1.36734,8.87493,35.43700,12.01782,303.67474,298.06860,33.6555,5.7509
4,2020-07-11 00:00:04,395.34866,12,10,52.79694,-1.7811,-1.88294,781.83160,28.03595,11384.0,...,827.86560,0.0,1.36810,8.83838,35.45227,12.01782,303.22266,296.53137,33.6555,5.8547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478796,2020-08-10 10:59:56,387.27219,12,10,66.72057,-0.9331,-1.84479,781.87915,28.02389,880.0,...,944.84705,0.0,1.32843,15.17817,35.14710,11.79657,316.89453,296.54950,32.0000,6.6026
478797,2020-08-10 10:59:57,387.52774,12,10,66.72057,-0.9996,-1.84479,787.65070,28.02385,840.0,...,940.49835,0.0,1.32843,15.17344,35.13183,11.79657,315.59247,296.15161,32.0000,6.3894
478798,2020-08-10 10:59:58,387.47665,12,10,66.72057,-1.2560,-1.84479,788.50256,28.03085,792.0,...,935.71472,0.0,1.32919,15.16443,35.13183,11.79657,313.92865,293.40277,32.0000,6.2584
478799,2020-08-10 10:59:59,387.73221,12,10,66.72057,-1.4912,-1.84479,785.80316,28.02649,752.0,...,944.84705,0.0,1.32843,15.09001,35.14710,11.79657,315.61054,302.58972,32.0000,6.4150


- Train set는 공격을 받지 않은 평상시의 데이터고, 시간=time, C01~C79는 학습 데이터셋에 있는 모든 센서/액추에이터 필드
- 정규화는 C01~C79 만을 해야한다.
- VALID_COLUMNS_IN_TRAIN_DATASET은 학습 데이터셋에 있는 모든 센서/액추에이터 필드를 담고 있다.
- 학습 데이터셋에 존재하지 않는 필드가 테스트 데이터셋에 존재하는 경우가 있다.
- 학습 시 보지 못했던 필드에 대해서 테스트를 할 수 없으므로 학습 데이터셋을 기준으로 필드 이름을 얻는다.

In [8]:
TIMESTAMP_FIELD = "time"
IDSTAMP_FIELD = 'id'
ATTACK_FIELD = "attack"
VALID_COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop([TIMESTAMP_FIELD])
VALID_COLUMNS_IN_TRAIN_DATASET

Index(['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
       'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40',
       'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50',
       'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60',
       'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70',
       'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79'],
      dtype='object')

#### Train set normalization

In [9]:
TAG_MIN = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()

TAG_MIN.head()

C01    355.98859
C02     12.00000
C03     10.00000
C04     46.17462
C05     -2.16030
dtype: float64

In [10]:
# 정규화 함수 생성 - MinMax Scaler
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

#### Train set smoothing
- 센서에서 발생하는 noise를 smoothing시키기 위해 exponential weighted function을 통과시킴
- ewm : 보다 최근의 값에 가중치를 두면서 이동평균 계산
- EWM_t = Xt*EP + EWM(t-1)*(1-EP), where EP(Exponential Percetage) = 2/(time+1) -https://m.blog.naver.com/PostView.nhn?blogId=gracekang7&logNo=221232491635&proxyReferer=https:%2F%2Fwww.google.com%2F

In [11]:
TRAIN_DF = normalize(TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()
TRAIN_DF.head(30)

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,0.378953,0.0,0.0,0.227071,0.372380,0.000230,0.386721,0.410567,0.784144,0.508049,...,0.584892,0.0,0.326835,0.254687,0.331076,0.916661,0.269393,0.265017,1.0,0.567254
1,0.378504,0.0,0.0,0.226596,0.353516,0.000161,0.399074,0.364415,0.794139,0.540538,...,0.592044,0.0,0.326835,0.254315,0.337223,0.916661,0.266791,0.251792,1.0,0.512135
2,0.378463,0.0,0.0,0.226789,0.318663,0.000154,0.393283,0.451729,0.803903,0.538802,...,0.595523,0.0,0.326387,0.255304,0.337777,0.916661,0.265266,0.254707,1.0,0.469622
3,0.378904,0.0,0.0,0.226808,0.238782,0.000154,0.393697,0.323289,0.813725,0.459532,...,0.596151,0.0,0.326343,0.257362,0.331746,0.916661,0.264379,0.253005,1.0,0.446285
4,0.380282,0.0,0.0,0.226810,0.165794,0.000154,0.412796,0.654203,0.823039,0.333541,...,0.598763,0.0,0.326786,0.256312,0.337229,0.916661,0.262757,0.247706,1.0,0.477489
5,0.379530,0.0,0.0,0.226810,0.124738,0.000154,0.398317,0.412616,0.832304,0.220924,...,0.600157,0.0,0.326830,0.257233,0.331692,0.916661,0.258607,0.248202,1.0,0.535436
6,0.380345,0.0,0.0,0.226810,0.122953,0.000154,0.433968,0.319390,0.840522,0.141019,...,0.601429,0.0,0.321441,0.258057,0.331138,0.916661,0.256413,0.246260,1.0,0.609982
7,0.380870,0.0,0.0,0.226810,0.182842,0.000154,0.392709,0.447405,0.848636,0.129432,...,0.595042,0.0,0.321350,0.258894,0.331083,0.916661,0.255150,0.244195,1.0,0.693586
8,0.380479,0.0,0.0,0.226810,0.263980,0.000154,0.388015,0.345183,0.856218,0.215276,...,0.594120,0.0,0.320893,0.260061,0.331077,0.916661,0.253368,0.242782,1.0,0.712285
9,0.380439,0.0,0.0,0.226810,0.332648,0.000223,0.391371,0.474158,0.863226,0.342822,...,0.594595,0.0,0.320847,0.260060,0.331077,0.916661,0.252944,0.241193,1.0,0.674545


#### data boundary check

In [12]:
# 결측, 0~1사이 값이 아닌게 있는지 점검
def boundary_check(df):
    x = np.array(df, dtype=np.float32)
    return np.any(x > 1.0), np.any(x < 0), np.any(np.isnan(x))

In [13]:
boundary_check(TRAIN_DF)

(False, False, False)

### RNN Model Fitting(GRU Cells)
- Train set에는 정상데이터만 있으므로 unsupervised learning 해야함
- 모델이 출력하는 예측값과 실제로 들어온 값의 차이가 크면 이상으로 간주함

#### 1. Sliding Window Algorithm
- baseline code에서는 윈도우 크기를 90초로 했었음
- 아래 코드에서는 윈도우 크기를 60초로, stride는 1로 지정하여 데이터 패턴을 자세히 더 자세히 기억하도록 함

In [14]:
WINDOW_GIVEN = 59
WINDOW_SIZE = 60

In [31]:
class HaiDataset:
    # 생성자
    def __init__(self, timestamps, df, stride=1, attacks=None):
        self.ts = np.array(timestamps)
        self.tag_values = np.array(df, dtype=np.float32)
        self.valid_idxs = []
        # slide window 유효성 확인
        for L in trange(len(self.ts) - WINDOW_SIZE + 1):
            R = L + WINDOW_SIZE - 1
            t_diff = parser.parse(self.ts[R]) - parser.parse(self.ts[L])
            self.valid_idxs.append(L)
        self.valid_idxs = np.array(self.valid_idxs, dtype=np.int32)[::stride]
        self.n_idxs = len(self.valid_idxs)
        print("# of valid windows : {}".format(self.n_idxs))
        
        if attacks is not None:
            self.attacks = np.array(attacks, dtype=np.float32)
            self.with_attack = True
        else:
            self.with_attack = False
    
    # 메서드 1
    def __len__(self):
        return self.n_idxs
    
    # 메서드 2 --> 나중에 given, answer부분 텐서로 변환해줘야 할수도 있음! 오류나면 나중에 고치기
    def __getitem__(self, idx):
        i = self.valid_idxs[idx]
        last = i + WINDOW_SIZE - 1 # slide end point
        item = {"attack" : self.attacks[last]} if self.with_attack else {}
        item["ts"] = self.ts[last] # i + WINDOW_SIZE - 1
        item["given"] = np.array(self.tag_values[i : i + WINDOW_GIVEN]) 
        item["answer"] = np.array(self.tag_values[last])
        return item

In [49]:
HAI_DATASET_TRAIN = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=1)
HAI_DATASET_TRAIN[0]


# of valid windows : 921544


{'ts': '2020-07-11 00:00:59',
 'given': array([[0.37895253, 0.        , 0.        , ..., 0.2650172 , 1.        ,
         0.5672542 ],
        [0.3785037 , 0.        , 0.        , ..., 0.25179192, 1.        ,
         0.512135  ],
        [0.37846327, 0.        , 0.        , ..., 0.25470728, 1.        ,
         0.46962237],
        ...,
        [0.3784445 , 0.        , 0.        , ..., 0.1795708 , 1.        ,
         0.6516667 ],
        [0.37756774, 0.        , 0.        , ..., 0.17767148, 1.        ,
         0.7075438 ],
        [0.37614608, 0.        , 0.        , ..., 0.17971386, 1.        ,
         0.65623736]], dtype=float32),
 'answer': array([3.7600392e-01, 0.0000000e+00, 0.0000000e+00, 2.2680973e-01,
        3.3855897e-01, 1.5367311e-04, 4.5900017e-01, 4.2067796e-01,
        9.8894030e-01, 5.2563941e-01, 5.2290523e-01, 1.6962849e-01,
        1.4555490e-01, 0.0000000e+00, 4.7653991e-01, 3.5710070e-01,
        5.2494091e-01, 0.0000000e+00, 0.0000000e+00, 1.7464979e-01,
     

#### GRU model fitting
- 3층 bidirectional GRU 사용
- Hidden cell의 크기 = 100
- Dropout X
- 모델이 윈도우의 가장 첫 번째 값과 RNN의 출력을 더해서 내보내도록 skip connection(forward 메소드의 return 문 참조)을 만듬

In [57]:
#########여기부터 코드 수정하기!

N_HIDDENS = 100
N_LAYERS = 3
BATCH_SIZE = 512

class StackedGRU():
    def __init__(self, n_tags):
        super().__init__()
        self.model = models.Sequential()
        self.model.add(
            layers.Bidirectional(layers.GRU(units= N_HIDDENS + N_LAYERS,
                                            input_shape= n_tags,
                                            activation='tanh', 
                                            recurrent_activation='sigmoid',
                                            dropout=0))) 
        self.fc = self.model.Dense(n_tags)

    def forward(self, x):
        x = x.transpose(0, 1)  # (batch, seq, params) -> (seq, batch, params)
        self.model.add(flatten())
        outs, _ = self.model(x)
        out = self.fc(outs[-1])
        return x[0] + out

In [58]:
MODEL = StackedGRU(n_tags=TRAIN_DF.shape[1])
MODEL()

TypeError: 'int' object is not iterable